# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355268133456                   -1.10    6.0    169ms
  2   -9.356788281164       -2.82       -1.43    1.0   79.0ms
  3   -9.357069873174       -3.55       -2.77    2.2    130ms
  4   -9.357119428882       -4.30       -2.99    8.1    497ms
  5   -9.357119850047       -6.38       -3.13    1.0   78.0ms
  6   -9.357120094225       -6.61       -3.29    1.0   77.4ms
  7   -9.357120238224       -6.84       -3.46    1.0   77.0ms
  8   -9.357120292533       -7.27       -3.64    1.0   77.2ms
  9   -9.357120316759       -7.62       -3.92    1.0   77.2ms
 10   -9.357120321911       -8.29       -4.15    4.2    121ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74756483190                   -0.97    6.8    494ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     7
│     1
│     8
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75918744840       -1.93       -1.32    2.6    294ms
  3   -18.79229351984       -1.48       -2.17    4.6    343ms
  4   -18.79261439860       -3.49       -2.50    6.0    440ms
  5   -18.79262184668       -5.13       -3.21    1.2    235ms
  6   -18.79262402891       -5.66       -3.61    5.6    421ms
  7   -18.79262416030       -6.88       -3.93    1.9    248ms
  8   -18.79262417484       -7.84       -4.37    2.0    244ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54722856486                   -0.84   10.2    1.79s
  2   -37.55785470018       -1.97       -1.22    2.2    746ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     15
│      6
│     17
│     16
│     26
│     18
│     12
│     10
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -37.56422819414       -2.20       -2.08   15.0    1.50s
  4   -37.56491300901       -3.16       -2.29    9.5    1.76s
  5   -37.56497630650       -4.20       -2.89    2.1    859ms
  6   -37.56498189054       -5.25       -3.32    7.2    1.42s
  7   -37.56498484988       -5.53       -3.74    2.9    999ms
  8   -37.56498518640       -6.47       -4.49    4.2    1.45s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355136400646                   -1.10    6.0    168ms
  2   -9.356814915889       -2.78       -1.91    1.0   67.7ms
  3   -9.357081013405       -3.57       -2.49    5.9    145ms
  4   -9.357102544633       -4.67       -2.62    1.9   86.7ms
  5   -9.357120074321       -4.76       -3.77    1.0   63.7ms
  6   -9.357120235234       -6.79       -3.83    8.6    190ms
  7   -9.357120306020       -7.15       -4.14    1.1   64.9ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     12
│      3
│     17
│      3
│     18
│     11
│     15
│      4
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54134134833                   -0.84   10.4    2.03s
  2   -37.54671249086       -2.27       -1.58    2.6    673ms
  3   -24.24583171629   +    1.12       -0.50    9.4    1.62s
  4   -37.46991663072        1.12       -1.53   10.1    2.01s
  5   -37.55078680720       -1.09       -1.82    2.2    786ms
  6   -37.36590491654   +   -0.73       -1.41    5.4    1.24s
  7   -37.50591591983       -0.85       -1.67    5.5    1.33s
  8   -37.56432555838       -1.23       -2.38    3.1    950ms
  9   -37.56473699814       -3.39       -2.55    4.4    1.20s
 10   -37.56465010536   +   -4.06       -2.47    2.6    1.29s
 11   -37.56494176416       -3.54    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.